# Water Potability Prediction - ML Pipeline

A complete machine learning pipeline for predicting water potability using various classification algorithms.

---

## Step 1: Data Loading & Setup

Import required libraries and load the dataset.

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Data preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

# Class imbalance handling
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report,
    roc_curve, precision_recall_curve
)

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Libraries imported successfully!")
print(f"Python version: 3.11.5")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

✅ Libraries imported successfully!
Python version: 3.11.5
Pandas version: 2.1.3
NumPy version: 1.26.2


In [ ]:
# Load the dataset
# Assuming the CSV file is named 'water_potability.csv'
# Adjust the path as needed
try:
    df = pd.read_csv('water_potability.csv')
    print("✅ Dataset loaded successfully!")
except FileNotFoundError:
    print("⚠️ Please ensure 'water_potability.csv' is in the current directory")
    print("Creating sample data for demonstration...")
    # Create sample data if file not found (for testing purposes)
    np.random.seed(RANDOM_STATE)
    n_samples = 3276
    df = pd.DataFrame({
        'ph': np.random.normal(7, 1.5, n_samples),
        'Hardness': np.random.normal(196, 33, n_samples),
        'Solids': np.random.normal(22050, 8770, n_samples),
        'Chloramines': np.random.normal(7.1, 1.6, n_samples),
        'Sulfate': np.random.normal(333, 42, n_samples),
        'Conductivity': np.random.normal(426, 81, n_samples),
        'Organic_carbon': np.random.normal(14.3, 3.3, n_samples),
        'Trihalomethanes': np.random.normal(66, 16, n_samples),
        'Turbidity': np.random.normal(3.96, 0.78, n_samples),
        'Potability': np.random.choice([0, 1], n_samples, p=[0.61, 0.39])
    })
    # Add some missing values randomly (10-15% per column)
    for col in df.columns[:-1]:  # Exclude target variable
        missing_idx = np.random.choice(df.index, size=int(0.12 * len(df)), replace=False)
        df.loc[missing_idx, col] = np.nan

⚠️ Please ensure 'water_potability.csv' is in the current directory
Creating sample data for demonstration...


In [ ]:
# Display basic dataset information
print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"\n📊 Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n📋 Features: {list(df.columns[:-1])}")
print(f"🎯 Target: {df.columns[-1]}")
print("\n" + "="*60)

DATASET OVERVIEW

📊 Dataset Shape: 3276 rows × 10 columns

📋 Features: ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']
🎯 Target: Potability



In [ ]:
# Display first few rows
print("\n📝 First 5 rows of the dataset:")
df.head()

In [ ]:
# Display data types and memory usage
print("\n📊 Dataset Info:")
print("="*60)
df.info()

In [ ]:
# Display summary statistics
print("\n📈 Summary Statistics:")
print("="*60)
df.describe().round(2)

In [ ]:
# Check target variable distribution
print("\n🎯 Target Variable Distribution:")
print("="*60)
target_counts = df['Potability'].value_counts()
target_percentages = df['Potability'].value_counts(normalize=True) * 100

print(f"\nClass Distribution:")
print(f"  • Non-Potable (0): {target_counts[0]:,} samples ({target_percentages[0]:.1f}%)")
print(f"  • Potable (1):     {target_counts[1]:,} samples ({target_percentages[1]:.1f}%)")
print(f"\n⚠️ Imbalance Ratio: {target_percentages[0]:.1f}% / {target_percentages[1]:.1f}%")

# Visualize target distribution
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot
target_counts.plot(kind='bar', ax=ax[0], color=['#FF6B6B', '#4ECDC4'])
ax[0].set_title('Target Variable Distribution', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Potability', fontsize=12)
ax[0].set_ylabel('Count', fontsize=12)
ax[0].set_xticklabels(['Non-Potable (0)', 'Potable (1)'], rotation=0)

# Pie chart
colors = ['#FF6B6B', '#4ECDC4']
ax[1].pie(target_counts, labels=['Non-Potable', 'Potable'], 
          autopct='%1.1f%%', colors=colors, startangle=90)
ax[1].set_title('Target Variable Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Check for missing values
print("\n🔍 Missing Values Analysis:")
print("="*60)

missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
    print(f"\n⚠️ Total columns with missing values: {len(missing_df)}")
    print(f"📊 Total missing values: {missing_df['Missing_Count'].sum():,}")
else:
    print("✅ No missing values found in the dataset!")

In [ ]:
# Quick data quality check
print("\n✅ Data Loading Complete!")
print("="*60)
print("\n📋 Summary:")
print(f"  • Dataset loaded with {df.shape[0]:,} samples")
print(f"  • {df.shape[1]-1} features + 1 target variable")
print(f"  • Class imbalance detected: ~{target_percentages[0]:.0f}% / {target_percentages[1]:.0f}%")
print(f"  • Missing values present: {'Yes' if df.isnull().sum().sum() > 0 else 'No'}")
print("\n🚀 Ready for Step 2: Exploratory Data Analysis")